<a href="https://colab.research.google.com/github/sushmit86/Statiistical_rethinking_jax/blob/main/sushmit_07__Ulysses_Compass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install packages that are not installed in colab
try:
  import google.colab
  !pip install watermark
  !pip install jaxopt
  !pip install networkx
  !pip install arviz
  !pip install causalgraphicalmodels
  !pip install daft-pgm
except:
  pass

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 3.5 MB/s eta 0:00:00


In [ ]:
#@title import library
import numpy as np
import scipy.stats as stats
import pandas as pd
import daft
import networkx as nx
import statsmodels.formula.api as smf
import jax
import io
import jax.numpy as jnp
from tensorflow_probability.substrates import jax as tfp_jax
from collections import OrderedDict
tfd = tfp_jax.distributions
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import jaxopt
from jaxopt import ScipyMinimize
import arviz
import itertools
import collections
import arviz as az
key = jax.random.PRNGKey(42)

def marginal_test_results(model, results, nll,credib_interval = 0.89, num_of_samples = 10000):
  '''
  model: tfp model like
  results: results from optimizing
  nll: negative log likelihood function
  credib_interval: the interval we are interested in
  num_of_samples: number of samples we want to get from the posterior distribution
  '''
  key = jax.random.PRNGKey(42)
  inv_hessian_matrix = jnp.linalg.inv(jax.hessian(nll)(results.params))
  mean_list = results.params
  sd_list = jnp.diagonal(inv_hessian_matrix)**(1/2)
  low_credib_interval = (1 - credib_interval)/2
  high_credib_interval = 1 - low_credib_interval
  parameter_list = []
  normal_dist = tfp_jax.distributions.Normal(loc=mean_list,scale =  sd_list)
  for _parameter in model.parameters['model']:
    if isinstance(model.parameters['model'][_parameter],tfp_jax.distributions.Distribution):
      try:
        _shape = model.parameters['model'][_parameter].event_shape[0]
        parameter_list.extend([_parameter + '_' +str(i) for i in range(0, _shape)])
      except:
        parameter_list.append(_parameter)

  df_result = pd.DataFrame({'mean': mean_list,
                           'sd':sd_list,
                            'low': normal_dist.quantile(low_credib_interval),
                            'high':tfp_jax.distributions.Normal(loc=mean_list,scale =  sd_list).quantile(high_credib_interval) },
                           index = parameter_list)
  post_dist = tfp_jax.distributions.MultivariateNormalFullCovariance(
    loc=results.params,covariance_matrix=inv_hessian_matrix )
  post_samples = post_dist.sample( sample_shape=num_of_samples,seed=key)
  df_post_samples = pd.DataFrame(post_samples,columns = parameter_list)
  return df_result,inv_hessian_matrix,df_post_samples
credib_interval = 0.89

In [ ]:
#@title 7.1
sppnames = ["afarensis","africanus","habilis","boisei", "rudolfensis","ergaster","sapiens"]
brainvolcc = [438 , 452 , 612, 521, 752, 871, 1350]
masskg = [37.0 , 35.5 , 34.5 , 41.5 , 55.5 , 61.0 , 53.5]
d = pd.DataFrame({'species': sppnames, 'brain': brainvolcc, 'mass': masskg})
d

,species,brain,mass
0,afarensis,438,37.0
1,africanus,452,35.5
2,habilis,612,34.5
3,boisei,521,41.5
4,rudolfensis,752,55.5
5,ergaster,871,61.0
6,sapiens,1350,53.5


In [ ]:
#@title 7.2
d['mass_std'] = (d.mass - np.mean(d.mass))/np.std(d.mass,ddof=1)
d['brain_std'] = d.brain/np.max(d.brain)
display(d)
brain_arr =jnp.array(d.brain_std)
mass_arr = jnp.array(d.mass_std)

,species,brain,mass,mass_std,brain_std
0,afarensis,438,37.0,-0.779467,0.324444
1,africanus,452,35.5,-0.917020,0.334815
2,habilis,612,34.5,-1.008722,0.453333
3,boisei,521,41.5,-0.366808,0.385926
4,rudolfensis,752,55.5,0.917020,0.557037
5,ergaster,871,61.0,1.421380,0.645185
6,sapiens,1350,53.5,0.733616,1.000000


In [ ]:
#@title 7.3
model7_1 = tfd.JointDistributionNamedAutoBatched(OrderedDict(
    alpha = tfd.Normal(loc = 0.5, scale = 1.0),
    beta = tfd.Normal(loc = 0.0, scale = 10.0),
    sigma = tfd.Normal(loc = 0.0, scale = 1.0),
    mu = lambda alpha,beta : tfd.Deterministic(alpha + beta*mass_arr),
    brain = lambda mu,sigma : tfd.Normal(loc = mu, scale = jnp.exp(sigma))
))
def nll7_1(parameters):
  samples = model7_1.sample(
      brain = brain_arr,
      alpha = parameters[0],
      beta = parameters[1],
      sigma = parameters[2],
      seed = jax.random.PRNGKey(42))
  return -1*model7_1.log_prob(samples)
optimize_fn = ScipyMinimize(fun = nll7_1, method = 'L-BFGS-B')
alpha_start = model7_1.parameters['model']['alpha'].mean()
beta_start = model7_1.parameters['model']['beta'].mean()
sigma_start = model7_1.parameters['model']['sigma'].mean()
results = optimize_fn.run(init_params = jnp.array([alpha_start,beta_start,sigma_start]))
df_result,cov_matrix, df_post_samples_m_7_1 = marginal_test_results(model7_1,results,nll7_1)
df_result.round(2)

,mean,sd,low,high
alpha,0.53,0.07,0.42,0.64
beta,0.17,0.07,0.05,0.29
sigma,-1.71,0.29,-2.18,-1.24


In [ ]:
leg_left_arr = jnp.array(d['leg_left'])
leg_right_arr = jnp.array(d['leg_right'])
height_arr = jnp.array(d['height'])
model6_1 = tfd.JointDistributionNamedAutoBatched(OrderedDict(
    alpha = tfd.Normal(loc=10.0,scale=100),
    bl = tfd.Normal(loc=2,scale=10),
    br = tfd.Normal(loc=2,scale=10),
    sigma = tfd.Exponential(rate = 1.0),
    mu = lambda alpha,bl,br: tfd.Deterministic(alpha + bl * leg_left_arr + br * leg_right_arr),
    h = lambda mu,sigma: tfd.Normal(loc=mu,scale=sigma)
))
def nll6_1(parameters):
  samples = model6_1.sample(
      h = height_arr,
      alpha = parameters[0],
      bl = parameters[1],
      br = parameters[2],
      sigma = parameters[3],
      seed = jax.random.PRNGKey(42))
  return -1* model6_1.log_prob(samples)

optimize_fn = ScipyMinimize(fun= nll6_1,method='L-BFGS-B')
alpha_start = model6_1.parameters['model']['alpha'].mean()
bl_start = model6_1.parameters['model']['bl'].mean()
br_start = model6_1.parameters['model']['br'].mean()
sigma_start = model6_1.parameters['model']['sigma'].mean()
results = optimize_fn.run(init_params=jnp.array([alpha_start,bl_start,br_start,sigma_start]))
df_result,cov_matrix,df_post_samples_m_6_1 = marginal_test_results(model6_1,results,nll6_1)

KeyError: 'leg_left'